# 🚀 Hierarchical RAG Pipeline - Ruhr Thesis (Colab Pro+ A100 Optimized)

**Optimized for Google Colab Pro+ with A100 GPU**

This notebook provides **50-100x faster** processing compared to CPU!

---

## 📋 Setup Checklist

Before running:
1. ✅ **Enable A100 GPU**: Runtime → Change runtime type → A100 GPU
2. ✅ **Upload files to Google Drive**: Your PDFs and data files
3. ✅ **Run cells in order**: Don't skip cells!

---

## ⚡ Expected Performance

| Task | CPU (Local) | A100 GPU (Colab) |
|------|-------------|------------------|
| 2 PDFs (78 nodes) | ~5-10 min | **~30 seconds** |
| 10 PDFs (~400 nodes) | ~30 min | **~2 minutes** |
| All 85 PDFs | ~2-3 hours | **~15-20 minutes** |

---

## 🔧 Step 1: Mount Google Drive & Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")

Mounted at /content/drive
✅ Google Drive mounted successfully!


In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🎉 GPU Available: {gpu_name}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    if "A100" in gpu_name:
        print("   ⚡ A100 GPU detected - OPTIMAL PERFORMANCE!")
else:
    print("⚠️  No GPU found. Go to Runtime → Change runtime type → Select A100 GPU")

🎉 GPU Available: NVIDIA A100-SXM4-40GB
   Memory: 42.47 GB
   ⚡ A100 GPU detected - OPTIMAL PERFORMANCE!


## 📦 Step 2: Install Dependencies

In [ ]:
%%capture
# Install required packages (suppressing output for cleaner notebook)
!pip install -q llama-index
!pip install -q llama-index-embeddings-huggingface
!pip install -q llama-index-vector-stores-chroma
!pip install -q chromadb
!pip install -q pymupdf
!pip install -q pandas
!pip install -q openpyxl
!pip install -q sentence-transformers

print("✅ All packages installed!")

## 📁 Step 3: Setup Paths

**IMPORTANT:** Update these paths to match your Google Drive structure!

In [ ]:
import os
from pathlib import Path

# ========================================
# 🔧 YOUR GOOGLE DRIVE PATH
# ========================================

# YOUR ACTUAL FOLDER NAME IN GOOGLE DRIVE
BASE_PATH = "/content/drive/MyDrive/PPE_Master_Thesis"

# PDF folders by phase
PDF_FOLDERS = {
    "phase1": f"{BASE_PATH}/Phase 1 - Theoretical Foundation",
    "phase2": f"{BASE_PATH}/Phase 2 - Sectoral & Business Transitions",
    "phase3": f"{BASE_PATH}/Phase 3 - Context & Case Studies",
    "phase4": f"{BASE_PATH}/Phase 4 - Methodology",
    "phase5": f"{BASE_PATH}/Phase 5 - Business Formation Literature"
}

# Quantitative data folder
QUANTITATIVE_DATA_PATH = f"{BASE_PATH}/Quantitative_Data"

DATA_SUBFOLDERS = {
    "landesdatenbank": f"{QUANTITATIVE_DATA_PATH}/processed_thesis_data_landesdatenbank",
    "inkar": f"{QUANTITATIVE_DATA_PATH}/inkar_datasets",
    "THESIS_DATA_FINAL": f"{QUANTITATIVE_DATA_PATH}/THESIS_DATA_FINAL",
    "comprehensive": f"{QUANTITATIVE_DATA_PATH}/comprehensive downloads"
}

# Output folder
OUTPUT_FOLDER = "/content/outputs"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Verify paths
print("=" * 80)
print("📂 VERIFYING PATHS")
print("=" * 80)

if os.path.exists(BASE_PATH):
    print(f"\n✅ Base path found: {BASE_PATH}\n")
else:
    print(f"\n❌ Base path NOT found: {BASE_PATH}")
    print(f"   Update BASE_PATH to match your Google Drive folder name!\n")

# Count PDFs
print("📄 PDF FILES:")
total_pdfs = 0
for phase, path in PDF_FOLDERS.items():
    if os.path.exists(path):
        pdf_count = sum(1 for root, dirs, files in os.walk(path)
                       for file in files if file.endswith('.pdf'))
        print(f"  ✅ {phase}: {pdf_count} PDFs")
        total_pdfs += pdf_count
    else:
        print(f"  ❌ {phase}: Not found")

print(f"  📊 Total PDFs: {total_pdfs}\n")

# Count data files
print("📊 QUANTITATIVE DATA:")
total_data = 0
for data_source, path in DATA_SUBFOLDERS.items():
    if os.path.exists(path):
        data_count = sum(1 for root, dirs, files in os.walk(path)
                        for file in files if file.endswith(('.csv', '.xlsx', '.xls')))
        print(f"  ✅ {data_source}: {data_count} data files")
        total_data += data_count
    else:
        print(f"  ❌ {data_source}: Not found")

print(f"  📊 Total data files: {total_data}\n")
print("=" * 80)


📂 VERIFYING PATHS

✅ Base path found: /content/drive/MyDrive/PPE_Master_Thesis

📄 PDF FILES:
  ✅ phase1: 22 PDFs
  ✅ phase2: 14 PDFs
  ✅ phase3: 25 PDFs
  ✅ phase4: 13 PDFs
  ✅ phase5: 11 PDFs
  📊 Total PDFs: 85

📊 QUANTITATIVE DATA:
  ✅ landesdatenbank: 9 data files
  ✅ inkar: 7 data files
  ✅ THESIS_DATA_FINAL: 45 data files
  ✅ comprehensive: 23 data files
  📊 Total data files: 84



## 🔨 Step 4: Initialize Pipeline Components

**GPU-Optimized Configuration:**
- Uses GPU for embedding generation
- Larger batch sizes for faster processing
- Optimized chunk sizes for A100 memory

In [ ]:
import sys
from pathlib import Path
from typing import List
import gc

from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
import chromadb

print("🔧 Initializing GPU-optimized embedding model...\n")

# Detect device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize embedding model (using same model as MCP system)
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-base-en-v1.5",  # Same as MCP system (768 dim instead of 384)
    device=device,
    embed_batch_size=64  # Large batch for GPU
)

# Set global settings - MATCHING MCP SYSTEM
Settings.embed_model = embed_model
Settings.chunk_size = 1000      # Smaller chunks like MCP system (was 1024)
Settings.chunk_overlap = 200    # Same overlap as MCP system

print(f"✅ Embedding model loaded")
print(f"   Device: {device.upper()}")
print(f"   Model: BAAI/bge-base-en-v1.5")
print(f"   Chunk size: 1000 chars (matching MCP system)")
print(f"   Chunk overlap: 200 chars")
print(f"   Batch size: 64")
print()

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path=f"{OUTPUT_FOLDER}/chromadb")
chroma_collection = chroma_client.get_or_create_collection("ppe_thesis_rag")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print("✅ Vector database initialized\n")

# GPU Memory stats
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"💾 GPU Memory: {allocated:.2f}GB / {total_mem:.0f}GB used")


🔧 Initializing GPU-optimized embedding model...



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding model loaded
   Device: CUDA
   Model: BAAI/bge-base-en-v1.5
   Chunk size: 1000 chars (matching MCP system)
   Chunk overlap: 200 chars
   Batch size: 64

✅ Vector database initialized

💾 GPU Memory: 0.44GB / 42GB used


## 📄 Step 5: PDF Processing Functions

In [ ]:
import fitz  # PyMuPDF
from datetime import datetime
from tqdm.auto import tqdm

def process_pdf(pdf_path: str) -> List[Document]:
    """Process a single PDF file"""
    documents = []

    try:
        doc = fitz.open(pdf_path)
        filename = Path(pdf_path).name

        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text()

            if text.strip():
                documents.append(Document(
                    text=text,
                    metadata={
                        "source": filename,
                        "page": page_num + 1,
                        "total_pages": len(doc),
                        "source_type": "pdf"
                    }
                ))

        doc.close()

    except Exception as e:
        print(f"❌ Error processing {pdf_path}: {e}")

    return documents

def process_multiple_pdfs(folder_path: str, max_pdfs: int = None) -> List[Document]:
    """Process multiple PDFs from a folder"""
    pdf_files = list(Path(folder_path).rglob("*.pdf"))

    if max_pdfs:
        pdf_files = pdf_files[:max_pdfs]

    print(f"\n📄 Processing {len(pdf_files)} PDFs from {folder_path}")

    all_documents = []

    for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
        docs = process_pdf(str(pdf_file))
        all_documents.extend(docs)

    print(f"✅ Created {len(all_documents)} document chunks")
    return all_documents

print("✅ PDF processing functions loaded")

✅ PDF processing functions loaded


## 🚀 Step 6: Run Quick Test (2 PDFs)

**This will take ~30-60 seconds on A100 GPU**

In [ ]:
import time

start_time = time.time()

print("="*80)
print("🚀 QUICK TEST: Processing 2 PDFs")
print("="*80)

# Process 2 PDFs from Phase 4
test_folder = PDF_FOLDERS["phase4"]
documents = process_multiple_pdfs(test_folder, max_pdfs=2)

print(f"\n🔨 Creating hierarchical chunks (matching MCP system)...")

# Create hierarchical chunks - MATCHING MCP SYSTEM
# MCP uses: Parent 2048, Child 1024, with BOTH indexed
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 1024, 512]  # 3 levels for more granular retrieval
)

nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

# INDEX ALL NODES (not just leaf nodes) - this matches MCP system behavior
# MCP indexes both parent and child chunks for better retrieval
all_nodes = nodes  # Use ALL nodes, not just get_leaf_nodes(nodes)

print(f"\n✅ Created {len(all_nodes)} total chunks (hierarchical)")
print(f"   (MCP system approach: indexing parent + child chunks)")

print(f"\n🔨 Building vector index (GPU-accelerated)...")

# Build index with ALL nodes
index = VectorStoreIndex(
    all_nodes,
    storage_context=storage_context,
    show_progress=True
)

elapsed = time.time() - start_time

print(f"\n" + "="*80)
print(f"✅ QUICK TEST COMPLETED!")
print(f"⏱️  Total time: {elapsed:.1f} seconds")
print(f"📊 Processed: {len(documents)} documents → {len(all_nodes)} chunks")
print("="*80)

# Memory stats
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    print(f"💾 GPU Memory used: {allocated:.2f}GB")


🚀 QUICK TEST: Processing 2 PDFs

📄 Processing 2 PDFs from /content/drive/MyDrive/PPE_Master_Thesis/Phase 4 - Methodology


Processing PDFs:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Created 80 document chunks

🔨 Creating hierarchical chunks (matching MCP system)...


Parsing documents into nodes:   0%|          | 0/80 [00:00<?, ?it/s]


✅ Created 315 total chunks (hierarchical)
   (MCP system approach: indexing parent + child chunks)

🔨 Building vector index (GPU-accelerated)...


Generating embeddings:   0%|          | 0/315 [00:00<?, ?it/s]


✅ QUICK TEST COMPLETED!
⏱️  Total time: 12.3 seconds
📊 Processed: 80 documents → 315 chunks
💾 GPU Memory used: 0.45GB


## 🔍 Step 7: Test Queries

Now let's test the RAG system!

In [ ]:
# Install Groq integration for LlamaIndex (FREE alternative to OpenAI)
!pip install -q llama-index-llms-groq

print("✅ Groq integration installed")

✅ Groq integration installed


In [ ]:
# (Optional) Install Google Gemini as backup
# !pip install -q llama-index-llms-gemini

print("✅ Dependencies ready")

✅ Dependencies ready


In [ ]:
import os
from google.colab import userdata
from llama_index.llms.groq import Groq
from llama_index.core import Settings

# Get API key from Colab secrets
# Add your Groq API key to Colab secrets as 'GROQ_API_KEY'
# Get free key at: https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# Set the LLM - Using Llama 3.3 70B (FREE via Groq)
Settings.llm = Groq(
    model="llama-3.3-70b-versatile",  # Options: "llama-3.3-70b-versatile", "llama-3.1-8b-instant", "mixtral-8x7b-32768"
    temperature=0.1,
    api_key=os.environ["GROQ_API_KEY"]
)

print("✅ Groq Llama 3.3 70B configured (FREE tier)")
print("   Model: llama-3.3-70b-versatile")
print("   Rate limit: 30 requests/minute on free tier")

✅ Groq Llama 3.3 70B configured (FREE tier)
   Model: llama-3.3-70b-versatile
   Rate limit: 30 requests/minute on free tier


In [ ]:
# Create query engine
query_engine = index.as_query_engine(
    similarity_top_k=5,
    response_mode="compact"
)

print("✅ Query engine ready!\n")

# Test query
test_query = "What is spatial econometrics?"
print(f"🔍 Query: {test_query}\n")

response = query_engine.query(test_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

# Show sources
print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')})")

✅ Query engine ready!

🔍 Query: What is spatial econometrics?

📝 Response:
Spatial econometrics is a sub-field of econometrics that deals with the analysis of spatial panel data, which involves modeling spatial interactions across spatial units and over time. It aims to capture the relationships between variables that are spatially correlated, taking into account the spatial structure of the data. Spatial econometrics includes various techniques, such as spatial autoregressive models, spatial error models, and spatial lag models, to estimate and diagnose the spatial effects in the data.

📚 Sources:
  1. Spatial Panel Data Models in R.pdf (Page 33)
  2. Spatial Panel Data Models in R.pdf (Page 5)
  3. Spatial Panel Data Models in R.pdf (Page 5)
  4. Spatial Panel Data Models in R.pdf (Page 12)


In [ ]:
# Create query engine
query_engine = index.as_query_engine(
    similarity_top_k=5,
    response_mode="compact"
)

print("✅ Query engine ready!\n")

# Test query
test_query = "Create a research question based on the topics and conclusions of the two pdf's "
print(f"🔍 Query: {test_query}\n")

response = query_engine.query(test_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

# Show sources
print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')})")

✅ Query engine ready!

🔍 Query: Create a research question based on the topics and conclusions of the two pdf's 

📝 Response:
What is the effectiveness of spatial panel data models, such as those described by Baltagi et al. (2003), in analyzing geo-nested data, and how can mixed-methods research approaches be used to validate the results of these models in various spatially dependent contexts?

📚 Sources:
  1. Spatial Panel Data Models in R.pdf (Page 22)
  2. Geo-Nested Analysis_ Mixed-Methods Research with Spatially Depend.pdf (Page 42)


## 🎯 Step 8: Interactive Query Cell

**Run this cell multiple times with different queries!**

In [ ]:
# Enter your query here
user_query = "What are institutional complementarities in varieties of capitalism?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: What are institutional complementarities in varieties of capitalism?

📝 Response:
There is no mention of institutional complementarities in varieties of capitalism in the provided context. The text discusses spatial dependence, interdependence among units, and mixed-methods research designs, but it does not address the topic of institutional complementarities in varieties of capitalism.

📚 Sources:
  1. Geo-Nested Analysis_ Mixed-Methods Research with Spatially Depend.pdf (Page 6) [Score: 0.406]
  2. Geo-Nested Analysis_ Mixed-Methods Research with Spatially Depend.pdf (Page 6) [Score: 0.403]
  3. Geo-Nested Analysis_ Mixed-Methods Research with Spatially Depend.pdf (Page 4) [Score: 0.389]


## 🎉 Step 9: Process ALL PDFs (Full Pipeline)

**This will process all 85 PDFs - takes ~15-20 minutes on A100**

⚠️ **Only run this when ready for full processing!**

In [ ]:
# Full pipeline - processes ALL PDFs
# This matches the MCP system's chunking strategy

start_time = time.time()

print("="*80)
print("🚀 FULL PIPELINE: Processing ALL PDFs")
print("="*80)

# Process all PDFs
all_documents = []
for phase, folder in PDF_FOLDERS.items():
    print(f"\n📂 Processing {phase}...")
    docs = process_multiple_pdfs(folder, max_pdfs=None)
    all_documents.extend(docs)
    gc.collect()  # Free memory

print(f"\n✅ Total documents: {len(all_documents)}")

# Create hierarchical chunks - MATCHING MCP SYSTEM
print(f"\n🔨 Creating hierarchical chunks (MCP-compatible)...")
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 1024, 512]  # 3 levels like MCP system
)

nodes = node_parser.get_nodes_from_documents(all_documents, show_progress=True)

# INDEX ALL NODES (parent + child) - matches MCP system
# This is the key difference: MCP indexes both hierarchical levels
all_nodes = nodes  # NOT get_leaf_nodes(nodes)

print(f"\n✅ Created {len(all_nodes)} total chunks")
print(f"   (Hierarchical: parent 2048 + child 1024 + leaf 512)")

# Build index
print(f"\n🔨 Building vector index (GPU-accelerated)...")
index = VectorStoreIndex(
    all_nodes,
    storage_context=storage_context,
    show_progress=True
)

elapsed = time.time() - start_time
print(f"\n" + "="*80)
print(f"✅ FULL PIPELINE COMPLETED!")
print(f"⏱️  Total time: {elapsed/60:.1f} minutes")
print(f"📊 Total: {len(all_documents)} documents → {len(all_nodes)} chunks")
print(f"📈 Expected: ~13,000+ chunks (matching MCP system)")
print("="*80)


🚀 FULL PIPELINE: Processing ALL PDFs

📂 Processing phase1...

📄 Processing 22 PDFs from /content/drive/MyDrive/PPE_Master_Thesis/Phase 1 - Theoretical Foundation


Processing PDFs:   0%|          | 0/22 [00:00<?, ?it/s]

✅ Created 1170 document chunks

📂 Processing phase2...

📄 Processing 14 PDFs from /content/drive/MyDrive/PPE_Master_Thesis/Phase 2 - Sectoral & Business Transitions


Processing PDFs:   0%|          | 0/14 [00:00<?, ?it/s]

✅ Created 314 document chunks

📂 Processing phase3...

📄 Processing 25 PDFs from /content/drive/MyDrive/PPE_Master_Thesis/Phase 3 - Context & Case Studies


...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/532 [00:00<?, ?it/s]


✅ FULL PIPELINE COMPLETED!
⏱️  Total time: 6.7 minutes
📊 Total: 2949 documents → 12820 chunks
📈 Expected: ~13,000+ chunks (matching MCP system)


## 💾 Step 10: Save Index to Google Drive

**Save your work so you don't have to rebuild the index!**

In [ ]:
# Save to Google Drive
save_path = f"{BASE_PATH}/Hierarchical_RAG_Pipeline/colab_index"
os.makedirs(save_path, exist_ok=True)

# Copy ChromaDB to Drive
import shutil
shutil.copytree(f"{OUTPUT_FOLDER}/chromadb", f"{save_path}/chromadb", dirs_exist_ok=True)

print(f"✅ Index saved to: {save_path}")
print("   You can reload this index in future sessions!")

✅ Index saved to: /content/drive/MyDrive/PPE_Master_Thesis/Hierarchical_RAG_Pipeline/colab_index
   You can reload this index in future sessions!


## 📋 Citation System - Summary & Usage Guide

### ✅ What You Now Have

**Three Export Files** (saved to Google Drive):
1. **`citation_database.csv`** - Spreadsheet with all 85 citations, sortable by author/year/phase
2. **`citation_library.md`** - Human-readable reference list organized by research phase
3. **`citations.json`** - Machine-readable database with full page content index

**Interactive Tools**:
- `verify_citation(author, year, page, quote)` - Verify if a citation exists
- Search functionality to find PDFs by keyword
- Page-level content index for all 2,900+ pages

---

### 🔧 How to Use This System

#### **When Writing Your Thesis:**

1. **Before citing**, verify the citation exists:
   ```python
   result = verify_citation("Hall", "2001", 355, "your quote here")
   print(result['status'])  # Should be "✅ VERIFIED"
   ```

2. **Get the proper APA citation**:
   - Open `citation_library.md` or `citation_database.csv`
   - Find the author/year
   - Copy the exact APA citation

3. **Never fabricate page numbers**:
   - Use the verification tool to find the exact page
   - If quote not found, the tool will show you the actual page content

#### **Fixing Chapter 2 Citations:**

The Citation Verification Report identified these issues:
- ❌ **RWI, 2018, p. 54** - Not in your PDFs, remove it
- ❌ **Hayter et al., 2003** - Misattributed, should be Martin & Sunley
- ❌ **Crouch et al., 2009, p. 654** - Page doesn't exist (paper is only ~25 pages)

**Action:** Use the verification tool to find correct citations!

---

### 📊 Statistics

- **Total PDFs indexed:** 85
- **Total pages:** ~2,900
- **Citations generated:** 85 APA references
- **Processing time:** ~10-15 minutes (one-time)
- **Storage location:** `/content/drive/MyDrive/PPE_Master_Thesis/Hierarchical_RAG_Pipeline/citations/`

---

### 💡 Tips

- **Always verify** before citing - don't trust memory or previous drafts
- **Use exact page numbers** from the verification tool
- **Check quote accuracy** - the tool will highlight mismatches
- **Save your work** - All files are in Google Drive for future sessions

---

**🎉 Your citations are now verifiable and properly formatted!**

In [ ]:
  # 🔄 RELOAD EXISTING INDEX
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

print("🔄 Loading existing index from Google Drive...")

  # Point to the chromadb FOLDER (not individual UUID folders)
SAVED_INDEX_PATH = f"{BASE_PATH}/Hierarchical_RAG_Pipeline/colab_index/chromadb"

  # Initialize embedding model - ENSURE THIS MATCHES THE MODEL USED FOR INDEX CREATION
embed_model = HuggingFaceEmbedding(
      model_name="BAAI/bge-base-en-v1.5", # Changed from bge-small-en-v1.5 to match the index creation model
      cache_folder="./models_cache"
  )

  # Connect to ChromaDB
chroma_client = chromadb.PersistentClient(path=SAVED_INDEX_PATH)

  # List all available collections (to see what you have)
print("\n📋 Available collections:")
collections = chroma_client.list_collections()
for coll in collections:
      print(f"   - {coll.name} (ID: {coll.id}, Count: {coll.count()})")

  # Get your specific collection by name
collection_name = "ppe_thesis_rag"  # This is the name from your notebook
chroma_collection = chroma_client.get_collection(collection_name)

print(f"\n✅ Loaded collection: {collection_name}")
print(f"   Documents: {chroma_collection.count()}")

  # Create vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

  # Load the index
index = VectorStoreIndex.from_vector_store(
      vector_store=vector_store,
      embed_model=embed_model
  )

print(f"✅ Index loaded and ready for queries!")

🔄 Loading existing index from Google Drive...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


📋 Available collections:
   - ppe_thesis_rag (ID: 91285782-8931-4d86-ad7a-b8ec3ad25940, Count: 13135)

✅ Loaded collection: ppe_thesis_rag
   Documents: 13135
✅ Index loaded and ready for queries!


In [ ]:
"""
Step 1: PDF Metadata Extraction
Extract citation information from all 85 PDFs
"""

import fitz  # PyMuPDF
from pathlib import Path
import re
from typing import Dict, Optional
import json

def extract_pdf_metadata(pdf_path: str) -> Dict:
    """Extract metadata from a single PDF"""
    try:
        doc = fitz.open(pdf_path)
        metadata = doc.metadata
        filename = Path(pdf_path).stem

        # Extract metadata fields
        citation_data = {
            "filename": Path(pdf_path).name,
            "filepath": pdf_path,
            "title": metadata.get("title", ""),
            "author": metadata.get("author", ""),
            "subject": metadata.get("subject", ""),
            "keywords": metadata.get("keywords", ""),
            "creator": metadata.get("creator", ""),
            "producer": metadata.get("producer", ""),
            "page_count": len(doc),
            "raw_filename": filename
        }

        doc.close()
        return citation_data

    except Exception as e:
        print(f"❌ Error extracting metadata from {pdf_path}: {e}")
        return None

# Extract metadata from all PDFs
print("="*80)
print("📚 EXTRACTING PDF METADATA FROM 85 PDFs")
print("="*80)

all_pdf_metadata = []

for phase, folder in PDF_FOLDERS.items():
    if os.path.exists(folder):
        pdf_files = list(Path(folder).rglob("*.pdf"))
        print(f"\n📂 {phase}: Processing {len(pdf_files)} PDFs...")

        for pdf_file in tqdm(pdf_files, desc=f"Extracting {phase}"):
            metadata = extract_pdf_metadata(str(pdf_file))
            if metadata:
                metadata["phase"] = phase
                all_pdf_metadata.append(metadata)

print(f"\n✅ Extracted metadata from {len(all_pdf_metadata)} PDFs")
print(f"📊 Sample metadata fields: {list(all_pdf_metadata[0].keys())}")

📚 EXTRACTING PDF METADATA FROM 85 PDFs

📂 phase1: Processing 22 PDFs...


Extracting phase1:   0%|          | 0/22 [00:00<?, ?it/s]


📂 phase2: Processing 14 PDFs...


Extracting phase2:   0%|          | 0/14 [00:00<?, ?it/s]


📂 phase3: Processing 25 PDFs...


...


📂 phase4: Processing 13 PDFs...


Extracting phase4:   0%|          | 0/13 [00:00<?, ?it/s]


📂 phase5: Processing 11 PDFs...


Extracting phase5:   0%|          | 0/11 [00:00<?, ?it/s]


✅ Extracted metadata from 85 PDFs
📊 Sample metadata fields: ['filename', 'filepath', 'title', 'author', 'subject', 'keywords', 'creator', 'producer', 'page_count', 'raw_filename', 'phase']


In [ ]:
"""
Step 2: Generate APA Citations Using Groq (FREE Alternative to Claude)
Parse metadata and filenames to create proper citations
"""

from llama_index.llms.groq import Groq
import time
import os

def generate_apa_citation_with_groq(metadata: Dict) -> Dict:
    """Use Groq (Llama 3.3 70B) to generate APA citation from metadata"""

    # Create prompt for Llama
    prompt = f"""Given this PDF metadata, generate a proper APA 7th edition citation.

PDF Metadata:
- Filename: {metadata['filename']}
- Title (from metadata): {metadata['title'] or 'Not available'}
- Author (from metadata): {metadata['author'] or 'Not available'}
- Pages: {metadata['page_count']}

Instructions:
1. If metadata has author/title/year, use it
2. If metadata is missing, parse the filename to extract:
   - Author name(s) (usually at start or after year)
   - Year (usually 4 digits like 2001, 2023, etc.)
   - Title (remaining text, convert underscores to spaces)
3. Determine source type (journal article, book, report, working paper)
4. Generate proper APA citation

Return ONLY a valid JSON object with these fields:
{{
  "authors": "Last, F. M., & Last2, F. M.",
  "year": "2001",
  "title": "Full title of the work",
  "source_type": "journal" or "book" or "report" or "working_paper",
  "journal": "Journal Name (if applicable)",
  "apa_citation": "Full APA formatted citation"
}}

Example filenames:
- "Varieties_of_Capitalism_hall_soskice.pdf" → Hall, P. A., & Soskice, D. (year from metadata)
- "2012_Thelen_Varieties_Liberalization.pdf" → Thelen, K. (2012)
- "foster-thelen-2024-coordination-rights.pdf" → Foster, A., & Thelen, K. (2024)

Be precise and follow APA 7th edition exactly. Return ONLY the JSON, no other text."""

    try:
        llm = Groq(
            model="llama-3.3-70b-versatile",
            temperature=0.1,
            api_key=os.environ.get("GROQ_API_KEY")
        )
        response = llm.complete(prompt)

        # Parse JSON response
        json_str = response.text.strip()
        # Extract JSON if wrapped in markdown code blocks
        if "```json" in json_str:
            json_str = json_str.split("```json")[1].split("```")[0].strip()
        elif "```" in json_str:
            json_str = json_str.split("```")[1].split("```")[0].strip()

        citation_info = json.loads(json_str)
        return citation_info

    except Exception as e:
        print(f"⚠️  Error generating citation for {metadata['filename']}: {e}")
        return {
            "authors": "Unknown",
            "year": "n.d.",
            "title": metadata['filename'],
            "source_type": "unknown",
            "journal": "",
            "apa_citation": f"Unknown. (n.d.). {metadata['filename']}."
        }

# Generate citations for all PDFs
print("="*80)
print("🤖 GENERATING APA CITATIONS WITH GROQ (Llama 3.3 70B)")
print("="*80)
print("⏱️  This may take 15-20 minutes for 85 PDFs (rate limited to 30 req/min)...\n")

citation_database = []
batch_size = 5  # Process in small batches to avoid rate limits

for i in tqdm(range(0, len(all_pdf_metadata), batch_size), desc="Generating citations"):
    batch = all_pdf_metadata[i:i+batch_size]

    for metadata in batch:
        citation_info = generate_apa_citation_with_groq(metadata)

        # Combine metadata with citation info
        complete_entry = {
            **metadata,
            **citation_info
        }
        citation_database.append(complete_entry)

    # Delay to respect Groq free tier rate limits (30 req/min)
    time.sleep(2)

print(f"\n✅ Generated {len(citation_database)} citations")
print("\n📋 Sample citation:")
print(f"   {citation_database[0]['apa_citation']}")


🤖 GENERATING APA CITATIONS WITH GROQ (Llama 3.3 70B)
⏱️  This may take 15-20 minutes for 85 PDFs (rate limited to 30 req/min)...



Generating citations:   0%|          | 0/17 [00:00<?, ?it/s]


✅ Generated 85 citations

📋 Sample citation:
   (Just Transition for Regions and Generations Experiences from Structural Change in the Ruhr Area, n.d.)


In [ ]:
"""
Step 3: Build Page-Level Content Index
Create searchable index mapping citations to page content
"""

def build_page_content_index(citation_database: list) -> Dict:
    """Build index mapping filename → page → content"""

    page_index = {}

    print("="*80)
    print("📖 BUILDING PAGE-LEVEL CONTENT INDEX")
    print("="*80)

    for entry in tqdm(citation_database, desc="Indexing pages"):
        filename = entry['filename']
        filepath = entry['filepath']

        try:
            doc = fitz.open(filepath)
            page_index[filename] = {
                "citation": entry['apa_citation'],
                "authors": entry['authors'],
                "year": entry['year'],
                "pages": {}
            }

            # Extract text from each page
            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()

                if text.strip():
                    page_index[filename]["pages"][page_num + 1] = {
                        "text": text,
                        "word_count": len(text.split())
                    }

            doc.close()

        except Exception as e:
            print(f"⚠️  Error indexing {filename}: {e}")

    print(f"\n✅ Indexed {len(page_index)} PDFs with full page content")

    return page_index

# Build the index
page_content_index = build_page_content_index(citation_database)

# Calculate total pages indexed
total_pages = sum(len(pdf_data["pages"]) for pdf_data in page_content_index.values())
print(f"📊 Total pages indexed: {total_pages:,}")

📖 BUILDING PAGE-LEVEL CONTENT INDEX


Indexing pages:   0%|          | 0/85 [00:00<?, ?it/s]


✅ Indexed 82 PDFs with full page content
📊 Total pages indexed: 2,894


In [ ]:
"""
Step 4: Citation Verification Tool
Verify citations against actual PDF content
"""

import difflib

def find_pdf_by_author_year(author_keyword: str, year: str, citation_database: list) -> Optional[Dict]:
    """Find PDF by author name and year"""
    for entry in citation_database:
        if (year in entry['year'] and
            author_keyword.lower() in entry['authors'].lower()):
            return entry
    return None

def verify_citation(author: str, year: str, page: int, quote: str = None) -> Dict:
    """
    Verify if a citation exists in the PDF collection

    Args:
        author: Author last name (e.g., "Hall", "Soskice")
        year: Publication year (e.g., "2001")
        page: Page number cited
        quote: Optional quote to verify (partial match OK)

    Returns:
        Dict with verification status and details
    """

    # Find the PDF
    pdf_entry = find_pdf_by_author_year(author, year, citation_database)

    if not pdf_entry:
        return {
            "status": "❌ NOT FOUND",
            "message": f"No PDF found matching author '{author}' and year '{year}'",
            "citation": None,
            "page_content": None
        }

    filename = pdf_entry['filename']

    # Check if PDF is in page index
    if filename not in page_content_index:
        return {
            "status": "❌ NOT INDEXED",
            "message": f"PDF found but not indexed: {filename}",
            "citation": pdf_entry['apa_citation'],
            "page_content": None
        }

    pdf_data = page_content_index[filename]

    # Check if page exists
    if page not in pdf_data['pages']:
        max_page = max(pdf_data['pages'].keys())
        return {
            "status": "⚠️ PAGE OUT OF RANGE",
            "message": f"Page {page} not found. PDF has {max_page} pages.",
            "citation": pdf_data['citation'],
            "available_pages": f"1-{max_page}",
            "page_content": None
        }

    page_text = pdf_data['pages'][page]['text']

    # If quote provided, verify it exists
    if quote:
        # Normalize quote and page text for comparison
        quote_normalized = ' '.join(quote.lower().split())
        text_normalized = ' '.join(page_text.lower().split())

        if quote_normalized in text_normalized:
            return {
                "status": "✅ VERIFIED",
                "message": f"Quote found on page {page}",
                "citation": pdf_data['citation'],
                "page": page,
                "page_content": page_text[:500] + "..." if len(page_text) > 500 else page_text,
                "quote_match": True
            }
        else:
            # Try fuzzy matching
            similarity = difflib.SequenceMatcher(None, quote_normalized, text_normalized).ratio()

            if similarity > 0.6:
                return {
                    "status": "⚠️ PARTIAL MATCH",
                    "message": f"Quote not found exactly, but page content is {similarity:.1%} similar",
                    "citation": pdf_data['citation'],
                    "page": page,
                    "page_content": page_text[:500] + "..." if len(page_text) > 500 else page_text,
                    "quote_match": False,
                    "similarity": f"{similarity:.1%}"
                }
            else:
                return {
                    "status": "❌ QUOTE NOT FOUND",
                    "message": f"Quote not found on page {page} (similarity: {similarity:.1%})",
                    "citation": pdf_data['citation'],
                    "page": page,
                    "page_content": page_text[:500] + "..." if len(page_text) > 500 else page_text,
                    "quote_match": False
                }
    else:
        # No quote to verify, just confirm page exists
        return {
            "status": "✅ PAGE EXISTS",
            "message": f"Page {page} exists in PDF",
            "citation": pdf_data['citation'],
            "page": page,
            "page_content": page_text[:500] + "..." if len(page_text) > 500 else page_text
        }

print("✅ Citation verification tool ready!")
print("\n📝 Example usage:")
print('   result = verify_citation("Hall", "2001", 355, "firms are often embedded")')
print('   print(result["status"])')
print('   print(result["citation"])')

✅ Citation verification tool ready!

📝 Example usage:
   result = verify_citation("Hall", "2001", 355, "firms are often embedded")
   print(result["status"])
   print(result["citation"])


In [ ]:
"""
Step 5: Export Citation Database (CSV, Markdown, JSON)
"""

import pandas as pd
import json
from datetime import datetime

# Create output directory
output_dir = f"{BASE_PATH}/Hierarchical_RAG_Pipeline/citations"
os.makedirs(output_dir, exist_ok=True)

print("="*80)
print("💾 EXPORTING CITATION DATABASE")
print("="*80)

# ==========================================
# 1. CSV Export
# ==========================================
print("\n📊 Creating CSV export...")

csv_data = []
for entry in citation_database:
    csv_data.append({
        "Author": entry['authors'],
        "Year": entry['year'],
        "Title": entry['title'],
        "Source_Type": entry['source_type'],
        "Journal": entry.get('journal', ''),
        "Pages": entry['page_count'],
        "Phase": entry['phase'],
        "Filename": entry['filename'],
        "APA_Citation": entry['apa_citation']
    })

df = pd.DataFrame(csv_data)
csv_path = f"{output_dir}/citation_database.csv"
df.to_csv(csv_path, index=False, encoding='utf-8')
print(f"   ✅ Saved: {csv_path}")

# ==========================================
# 2. Markdown Export
# ==========================================
print("\n📝 Creating Markdown export...")

md_content = f"""# Citation Library - {len(citation_database)} Sources

**Generated:** {datetime.now().strftime("%Y-%m-%d %H:%M")}

This document contains all properly formatted APA citations for the 85 PDFs in your thesis research collection.

---

"""

# Group by phase
phases = {
    "phase1": "Phase 1: Theoretical Foundation",
    "phase2": "Phase 2: Sectoral & Business Transitions",
    "phase3": "Phase 3: Context & Case Studies",
    "phase4": "Phase 4: Methodology",
    "phase5": "Phase 5: Business Formation Literature"
}

for phase_key, phase_title in phases.items():
    phase_entries = [e for e in citation_database if e['phase'] == phase_key]

    if phase_entries:
        md_content += f"\n## {phase_title} ({len(phase_entries)} sources)\n\n"

        # Sort by author
        phase_entries.sort(key=lambda x: x['authors'])

        for i, entry in enumerate(phase_entries, 1):
            md_content += f"{i}. **{entry['apa_citation']}**\n"
            md_content += f"   - File: `{entry['filename']}`\n"
            md_content += f"   - Pages: {entry['page_count']}\n"
            md_content += f"   - Type: {entry['source_type']}\n\n"

md_path = f"{output_dir}/citation_library.md"
with open(md_path, 'w', encoding='utf-8') as f:
    f.write(md_content)
print(f"   ✅ Saved: {md_path}")

# ==========================================
# 3. JSON Export (with page index)
# ==========================================
print("\n🔧 Creating JSON export...")

json_data = {
    "metadata": {
        "generated": datetime.now().isoformat(),
        "total_pdfs": len(citation_database),
        "total_pages": sum(e['page_count'] for e in citation_database)
    },
    "citations": {}
}

for entry in citation_database:
    filename = entry['filename']

    # Add citation data
    json_data["citations"][filename] = {
        "apa": entry['apa_citation'],
        "authors": entry['authors'],
        "year": entry['year'],
        "title": entry['title'],
        "source_type": entry['source_type'],
        "journal": entry.get('journal', ''),
        "phase": entry['phase'],
        "pages": entry['page_count'],
        "filepath": entry['filepath']
    }

    # Add page content if available
    if filename in page_content_index:
        json_data["citations"][filename]["page_map"] = {
            str(page_num): {
                "word_count": page_data['word_count'],
                "preview": page_data['text'][:200] + "..." if len(page_data['text']) > 200 else page_data['text']
            }
            for page_num, page_data in page_content_index[filename]['pages'].items()
        }

json_path = f"{output_dir}/citations.json"
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False)
print(f"   ✅ Saved: {json_path}")

# ==========================================
# Summary
# ==========================================
print("\n" + "="*80)
print("✅ EXPORT COMPLETE!")
print("="*80)
print(f"\n📁 All files saved to: {output_dir}/")
print(f"\n📊 Files created:")
print(f"   1. citation_database.csv     ({len(citation_database)} rows)")
print(f"   2. citation_library.md       (Human-readable)")
print(f"   3. citations.json            (Machine-readable with page index)")
print(f"\n💾 Total size: ~{sum(e['page_count'] for e in citation_database):,} pages indexed")
print("="*80)

💾 EXPORTING CITATION DATABASE

📊 Creating CSV export...
   ✅ Saved: /content/drive/MyDrive/PPE_Master_Thesis/Hierarchical_RAG_Pipeline/citations/citation_database.csv

📝 Creating Markdown export...
   ✅ Saved: /content/drive/MyDrive/PPE_Master_Thesis/Hierarchical_RAG_Pipeline/citations/citation_library.md

🔧 Creating JSON export...
   ✅ Saved: /content/drive/MyDrive/PPE_Master_Thesis/Hierarchical_RAG_Pipeline/citations/citations.json

✅ EXPORT COMPLETE!

📁 All files saved to: /content/drive/MyDrive/PPE_Master_Thesis/Hierarchical_RAG_Pipeline/citations/

📊 Files created:
   1. citation_database.csv     (85 rows)
   2. citation_library.md       (Human-readable)
   3. citations.json            (Machine-readable with page index)

💾 Total size: ~2,994 pages indexed


In [ ]:
"""
Step 6: Test Citation Verification
Test the verification tool with known citations from your Chapter 2
"""

print("="*80)
print("🧪 TESTING CITATION VERIFICATION")
print("="*80)

# Test cases from Chapter 2 Citation Verification Report
test_cases = [
    {
        "name": "Hall & Soskice 2001, p. 355 - Non-market coordination quote",
        "author": "Hall",
        "year": "2001",
        "page": 355,
        "quote": "firms are often embedded in arrangements that involve more extensive relational"
    },
    {
        "name": "Hall & Gingerich 2009, p. 4 - Institutional complementarities",
        "author": "Hall",
        "year": "2009",
        "page": 4,
        "quote": "One set of institutions is said to be complementary to another"
    },
    {
        "name": "Foster & Thelen 2024, p. 1 - Competition law",
        "author": "Foster",
        "year": "2024",
        "page": 1,
        "quote": "competition law"
    },
    {
        "name": "Crouch et al. 2009, p. 654 - KNOWN BAD PAGE NUMBER",
        "author": "Crouch",
        "year": "2009",
        "page": 654,
        "quote": None  # This should fail - page doesn't exist
    }
]

print("\n🔍 Running test cases...\n")

for test in test_cases:
    print(f"{'='*60}")
    print(f"Test: {test['name']}")
    print(f"{'='*60}")

    result = verify_citation(
        author=test['author'],
        year=test['year'],
        page=test['page'],
        quote=test['quote']
    )

    print(f"Status: {result['status']}")
    print(f"Message: {result['message']}")

    if result.get('citation'):
        print(f"Citation: {result['citation'][:100]}...")

    if result.get('page_content'):
        print(f"Page preview: {result['page_content'][:150]}...")

    print()

print("="*80)
print("✅ Testing complete!")
print("="*80)

🧪 TESTING CITATION VERIFICATION

🔍 Running test cases...

Test: Hall & Soskice 2001, p. 355 - Non-market coordination quote
Status: ❌ NOT FOUND
Message: No PDF found matching author 'Hall' and year '2001'

Test: Hall & Gingerich 2009, p. 4 - Institutional complementarities
Status: ❌ QUOTE NOT FOUND
Message: Quote not found on page 4 (similarity: 0.1%)
Citation: Hall, P. A., & Gingerich, D. W. (2009). Varieties of capitalism and institutional complementarities ...
Page preview: THE VARIETIES-OF-CAPITALISM APPROACH
In contrast to the literature focused on national labour movements, varieties-of-capitalism
analyses assume that ...

Test: Foster & Thelen 2024, p. 1 - Competition law
Status: ✅ VERIFIED
Message: Quote found on page 1
Citation: Foster, A., & Thelen, K. (2024). Coordination rights, competition law and varieties of capitalism....
Page preview: Article
Comparative Political Studies
2025, Vol. 58(6) 1199–1237
© The Author(s) 2024
Article reuse guidelines:
sagepub.com/journals-per

In [ ]:
# ==========================================
# SEARCH ALL CITATIONS
# ==========================================
# Search for PDFs by keyword in title, author, or content

search_keyword = "varieties of capitalism"  # Change this to search

print("="*80)
print(f"🔎 SEARCHING FOR: '{search_keyword}'")
print("="*80)

matches = []

for entry in citation_database:
    # Search in title, authors, and keywords
    search_in = f"{entry['title']} {entry['authors']} {entry.get('keywords', '')}".lower()

    if search_keyword.lower() in search_in:
        matches.append(entry)

print(f"\n✅ Found {len(matches)} matching PDFs:\n")

for i, match in enumerate(matches, 1):
    print(f"{i}. {match['apa_citation']}")
    print(f"   File: {match['filename']}")
    print(f"   Pages: {match['page_count']}")
    print(f"   Phase: {match['phase']}\n")

if not matches:
    print("❌ No matches found. Try a different keyword.")

print("="*80)

🔎 SEARCHING FOR: 'varieties of capitalism'

✅ Found 8 matching PDFs:

1. Crouch, I. G. P. M. C. (n.d.). Regional and sectoral varieties of capitalism
   File: Regional_and_Sectoral_Varieties_of_Capitalism_crouch.pdf
   Pages: 30
   Phase: phase1

2. Movahed, M. (2023). Varieties of capitalism and income inequality.
   File: movahed-2023-varieties-of-capitalism-and-income-inequality.pdf
   Pages: 38
   Phase: phase1

3. Foster, A., & Thelen, K. (2024). Coordination rights, competition law and varieties of capitalism.
   File: foster-thelen-2024-coordination-rights-competition-law-and-varieties-of-capitalism (1).pdf
   Pages: 39
   Phase: phase1

4. Hall, P. A., & Gingerich, D. W. (2009). Varieties of capitalism and institutional complementarities in the political economy: An empirical analysis.
   File: hallgingerich2009.pdf
   Pages: 34
   Phase: phase1

5. (An introduction to varieties of capitalism, n.d.)
   File: An_introduction_to_varieties_of_capitalism.pdf
   Pages: 68
   Phase: 

In [ ]:
# ==========================================
# VERIFY A CITATION
# ==========================================
# Change these values to verify your citations!

author_name = "Hall"        # Last name of author
pub_year = "2001"           # Publication year
page_number = 355           # Page number
quote_text = "firms are often embedded"  # Quote to verify (optional, set to None to skip)

# Run verification
result = verify_citation(author_name, pub_year, page_number, quote_text)

# Display results
print("="*80)
print(f"🔍 VERIFYING: {author_name} ({pub_year}), p. {page_number}")
print("="*80)
print(f"\nStatus: {result['status']}")
print(f"Message: {result['message']}\n")

if result.get('citation'):
    print(f"✅ APA Citation:\n   {result['citation']}\n")

if result.get('page_content'):
    print(f"📄 Page {page_number} content (first 400 chars):")
    print(f"   {result['page_content'][:400]}...\n")

if result.get('quote_match'):
    print("✅ Quote verified on this page!")
elif result.get('quote_match') == False:
    print("⚠️  Quote not found exactly - check page content above")

print("="*80)

🔍 VERIFYING: Hall (2001), p. 355

Status: ❌ NOT FOUND
Message: No PDF found matching author 'Hall' and year '2001'



## 📊 Performance Comparison

| Metric | Your Local CPU | Colab A100 GPU | Speedup |
|--------|----------------|----------------|----------|
| 2 PDFs (80 nodes) | ~10 minutes | **~30 seconds** | **20x faster** |
| Embedding speed | ~3-10 sec/node | **~0.05 sec/node** | **60-200x faster** |
| Memory efficient | Limited | 40GB GPU RAM | **Massive scale** |

---

## 🎓 Next Steps

1. ✅ **Quick test completed** - System works!
2. 🚀 **Run full pipeline** - Process all 85 PDFs (~15-20 min)
3. 💾 **Save index** - Never rebuild again!
4. 🔍 **Query your data** - Interactive research assistant ready!

---

## 📝 Sample Queries

```python
# Theory questions
"What are the key concepts in varieties of capitalism?"
"Explain institutional complementarities"

# Methodology questions  
"What spatial econometric methods are discussed?"
"How to analyze panel data?"

# Literature questions
"What studies discuss Ruhr industrial decline?"
"Recent research on just transitions"
```

---


In [ ]:
# Create query engine
query_engine = index.as_query_engine(
    similarity_top_k=5,
    response_mode="compact"
)

print("✅ Query engine ready!\n")

# Test query
test_query = "What is spatial econometrics?"
print(f"🔍 Query: {test_query}\n")

response = query_engine.query(test_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

# Show sources
print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')})")

✅ Query engine ready!

🔍 Query: What is spatial econometrics?

📝 Response:
Spatial econometrics is a field that deals with the analysis of spatial data, focusing on spatial lags, interaction effects, and spillover effects. It involves the use of econometric models to study the relationships between variables in different spatial locations, and to estimate the effects of these relationships on the variables of interest. Spatial econometric models can be used to answer questions such as how to interpret the outcomes of a spatial econometric model, how to estimate such a model, and how to select the appropriate spatial weights matrix and econometric model.

📚 Sources:
  1. gc_ws1819_Elhorst_presentation.pdf (Page 1)
  2. Spatial Panel Data Models in R.pdf (Page 33)


**Query Block 1: CME & Post-Industrial Transitions**

In [ ]:
# Enter your query here
user_query = "what are panel methods"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: what are panel methods

📝 Response:
Panel methods refer to techniques used for the estimation and testing of spatial panel data models, which involve the analysis of data that varies across both space and time. These methods are designed to account for the spatial relationships and correlations between observations, and can include the estimation of extra coefficients such as spatial lag terms and error correlation coefficients.

📚 Sources:
  1. Spatial Panel Data Models in R.pdf (Page 4) [Score: 0.507]
  2. Spatial Panel Data Models in R.pdf (Page 4) [Score: 0.498]


In [ ]:
# Enter your query here
user_query = "Explain institutional complementarities"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: Explain institutional complementarities

📝 Response:
Institutional complementarities refer to the idea that the presence or efficiency of one institution can increase the returns from or efficiency of another institution. In other words, two institutions can be said to be complementary if they work well together, enhancing each other's effectiveness. Conversely, institutions can also be substitutable, meaning that the absence or inefficiency of one institution can increase the returns to using another. This concept is important in understanding how different institutions interact and influence each other in a political economy, and how they can contribute to the overall performance of an economy.

📚 Sources:
  1. Varieties of Capitalism and Institutional Complementarities in the Political Economy_Hall_Gingerich.pdf (Page 37) [Score: 0.595]
  2. An_introduction_to_varieties_of_capitalism.pdf (Page 17) [Score: 0.591]
  3. Varieties of Capitalism and Institutional Complementariti

In [ ]:
# Enter your query here
user_query = "What are the key concepts in varieties of capitalism?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: What are the key concepts in varieties of capitalism?

📝 Response:
The key concepts in varieties of capitalism include coordinated market economies (CMEs) and liberal market economies (LMEs), which represent different ways to organize capitalism. These concepts emphasize the arrangements that define distinctive models of capitalism, including industrial relations institutions, financial arrangements, systems of vocational education and training, corporate governance, and social policy regimes. The approach also highlights the linkages across these institutions and how they shape policy and institutional preferences of economic actors. Additionally, the concept of national interests and how they are constructed for international negotiations is also a key aspect, with the organization of the political economy influencing the positions taken by nations in such negotiations.

📚 Sources:
  1. Varieties_of_Capitalism_hall_soskice.pdf (Page 236) [Score: 0.620]
  2. An_introduction_t

In [ ]:
# Enter your query here
user_query = "given the avaliable context what would this mean The research design employs panel logic—leveraging cross-sectional and temporal variation—without applying panel econometric methods"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: given the avaliable context what would this mean The research design employs panel logic—leveraging cross-sectional and temporal variation—without applying panel econometric methods

📝 Response:
The research design utilizes a panel structure, which involves analyzing data that has both cross-sectional and time-series components. However, it does not employ the typical methods used in panel econometrics, such as those that account for individual and time effects, or those that use specific estimation techniques like fixed or random effects models. Instead, the design focuses on exploiting the variation in the data across different cross-sections and over time, without applying the standard panel data methods. This approach allows for the examination of relationships and patterns in the data, but it may not fully account for the complexities and nuances of panel data, such as autocorrelation, heteroskedasticity, or unit effects.

📚 Sources:
  1. Spatial Panel Data Models in R.pd

In [ ]:
# Enter your query here
user_query =  "base don the contect on methodology Does panel login This aligns with the **explicitly exploratory and descriptive** objective: documenting variation rather than estimating effects, identifying patterns requiring explanation rather than testing hypotheses"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: base don the contect on methodology Does panel login This aligns with the **explicitly exploratory and descriptive** objective: documenting variation rather than estimating effects, identifying patterns requiring explanation rather than testing hypotheses

📝 Response:
The methodology discussed does align with an explicitly exploratory and descriptive objective. By considering spatial dependence in the preliminary analysis, researchers can identify patterns and document variation in the data, which can help guide further investigation and theory development. This approach prioritizes understanding the structure and nature of the data, rather than immediately testing hypotheses or estimating effects. By doing so, it provides a foundation for more robust and satisfactory results, and can help avoid model misspecification and omitted variable bias.

📚 Sources:
  1. Geo-Nested Analysis_ Mixed-Methods Research with Spatially Depend.pdf (Page 11) [Score: 0.476]


In [ ]:
# Enter your query here
user_query = "does anywhere in the context talk about coordination effectiveness proxies and how they are derived?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: does anywhere in the context talk about coordination effectiveness proxies and how they are derived?

📝 Response:
No, the context does not mention coordination effectiveness proxies or how they are derived. It discusses the concept of "more effective" coordination, which refers to coordination that leads to Pareto-superior equilibria, making at least some actors better off without making others worse off. However, it does not provide information on specific proxies or methods for measuring coordination effectiveness.

📚 Sources:
  1. Varieties_of_Capitalism_hall_soskice.pdf (Page 62) [Score: 0.521]
  2. An_introduction_to_varieties_of_capitalism.pdf (Page 46) [Score: 0.508]
  3. An_introduction_to_varieties_of_capitalism.pdf (Page 46) [Score: 0.502]
  4. An_introduction_to_varieties_of_capitalism.pdf (Page 45) [Score: 0.502]


In [ ]:
# Enter your query here
user_query = "What example variables are researchers supposed to select that serve as indirect indicators of coordination effectiveness"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: What example variables are researchers supposed to select that serve as indirect indicators of coordination effectiveness

📝 Response:
Researchers are supposed to select variables such as institutional measures that are commonly associated with one type of coordination or another, including indicators of support for strategic coordination and indicators of support for market coordination. These variables serve as indirect indicators of coordination effectiveness.

📚 Sources:
  1. Varieties of Capitalism and Institutional Complementarities in the Political Economy_Hall_Gingerich.pdf (Page 9) [Score: 0.549]
  2. Varieties_of_Capitalism_hall_soskice.pdf (Page 62) [Score: 0.535]
  3. hallgingerich2009.pdf (Page 6) [Score: 0.533]


##3.1_research_design

In [ ]:
# Enter your query here
user_query = "What are the specific strengths of using comparative case study design for analyzing post-industrial regional transitions like the Ruhr region? Why is it valuable to examine both successful and unsuccessful transformation cases?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: What are the specific strengths of using comparative case study design for analyzing post-industrial regional transitions like the Ruhr region? Why is it valuable to examine both successful and unsuccessful transformation cases?

📝 Response:
The comparative case study design offers several strengths for analyzing post-industrial regional transitions like the Ruhr region. Firstly, it allows for the examination of a sample that displays both similarities and differences across different criteria, which helps to draw out more general findings and conclusions. This approach enables the identification of positive and negative factors that may be isolated to a specific industry or region, as well as those that are more widely distributed.

By examining both successful and unsuccessful transformation cases, researchers can ascertain whether the factors that appear to explain success also appear to explain lack of success. This is valuable because it provides a more comprehensive unde

In [ ]:
# Enter your query here
user_query = "In mixed-methods research with spatially dependent data, what are the key advantages of combining Large-N quantitative analysis with Small-N qualitative case studies? How does this approach specifically address spatial dependence issues?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: In mixed-methods research with spatially dependent data, what are the key advantages of combining Large-N quantitative analysis with Small-N qualitative case studies? How does this approach specifically address spatial dependence issues?

📝 Response:
The key advantage of combining Large-N quantitative analysis with Small-N qualitative case studies is the ability to leverage the strengths of both approaches within a single unified framework. This mixed-methods design allows for the identification of broad patterns and trends through quantitative analysis, while also providing in-depth insights into specific contexts and cases through qualitative analysis. 

In the context of spatially dependent data, this approach is particularly useful as it enables researchers to address spatial dependence issues. If diagnostics indicate a spatial error process, the Small-N analysis can uncover spatially clustered omitted or unobserved variables, shedding light on "contextual effects". This h

In [ ]:
# Enter your query here
user_query = "In studies of the Ruhr region's economic transformation, what makes cities like Dortmund, Essen, Duisburg, and Bochum suitable as comparable cases? How do these cities share similar post-industrial contexts while showing variation in entrepreneurial outcomes?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: In studies of the Ruhr region's economic transformation, what makes cities like Dortmund, Essen, Duisburg, and Bochum suitable as comparable cases? How do these cities share similar post-industrial contexts while showing variation in entrepreneurial outcomes?

📝 Response:
The cities of Dortmund, Essen, Duisburg, and Bochum are suitable as comparable cases in studies of the Ruhr region's economic transformation due to their shared post-industrial context. All four cities have a strong industrial heritage, but have experienced significant economic restructuring in recent years. They have undergone similar processes of deindustrialization, leading to a decline in traditional industries such as coal mining and steel production.

Despite these similarities, the cities show variation in entrepreneurial outcomes. For example, Dortmund has a strong startup scene, with initiatives like the start2grow competition, while Essen is home to important players like ruhr:HUB and GründerAllianz

## 3.2_case_selection

In [ ]:
# Enter your query here
user_query = "In studies of business formation patterns in German regions, why is municipal-level analysis preferred over broader regional aggregation? What are the specific advantages of analyzing labor market regions or individual municipalities?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In spatial econometric research, how should researchers select cases to maximize outcome variation? What role do extreme spatial lag (rho) or spatial error (lambda) values play in identifying focal units for in-depth analysis?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In studies of German regional entrepreneurship patterns from 2002-2020, what historical context makes this time period suitable for analyzing post-reunification business formation dynamics? How does this timeframe capture both institutional stability and structural changes?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

## 3.3_data_measurement

In [ ]:
# Enter your query here
user_query = "In German regional studies, how is new business formation measured at the municipal level? What data sources distinguish between serial entrepreneurs (those who founded businesses previously) versus de-novo entrepreneurs (first-time founders)?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In empirical studies of coordinated market economies, what specific indicators measure institutional quality and coordination effectiveness? Which of these institutional measures can be operationalized at sub-national or regional levels?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: In empirical studies of coordinated market economies, what specific indicators measure institutional quality and coordination effectiveness? Which of these institutional measures can be operationalized at sub-national or regional levels?

📝 Response:
In empirical studies of coordinated market economies, specific indicators that measure institutional quality and coordination effectiveness include those related to labor relations and corporate governance. These may encompass variables such as the degree of wage bargaining coordination, the presence of powerful workforce representatives and business networks, and the extent to which firms adhere to consensual styles of decision-making. 

Some of these institutional measures can be operationalized at sub-national or regional levels, such as the degree of wage bargaining coordination, which can vary across different regions within a country. Additionally, the presence of business networks and the quality of labor relations can also

In [ ]:
# Enter your query here
user_query = "What indicators are typically included in regional competitiveness indices that measure economic dynamism? Which indicators specifically capture institutional quality versus pure economic outcomes like GDP?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In spatial econometric analysis of cross-sectional municipal data,what are the main approaches for constructing spatial weight matrices? What are the trade-offs between geographic contiguity, distance-based, and economic connectivity specifications?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "how are business formed in the Ruhr area? how have businesses been formed after the industrial decline"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

🔍 Query: how are business formed in the Ruhr area? how have businesses been formed after the industrial decline

📝 Response:
New economic sectors often grow out of old sectors in the Ruhr area. For example, the environmental economy was created by the mining industry due to increased environmental requirements. Companies in the same industry often concentrate locally, and adjustments to product ranges are usually made by the companies themselves in response to changes in demand. 

After the industrial decline, the Ruhr area underwent economic restructuring, which involved developing new programs, institutional restructuring, and a shift from heavy manufacturing to the service industry. This shift created new jobs, particularly in the service industry, and reskilling programs were implemented to retrain unemployed individuals and retain population in the region. Additionally, new businesses have been formed, such as those in the environmental economy, which have helped to drive economic

## 3.4_analytical_strategy

In [ ]:
# Enter your query here
user_query = "In spatial panel data models, what is the full specification for a model that includes both spatial lag effects (spillovers from neighboring units) and spatial error correlation? How are these spatial parameters interpreted?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In cross-sectional spatial analysis, why must researchers account for spatial dependence between observations? What biases occur when spatial relationships between units are ignored in the analysis?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In comparative case study research, how do researchers verify the temporal sequencing of events and processes across multiple cases? What role does process tracing play in identifying causal mechanisms?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In geo-nested analysis that combines quantitative and qualitative methods,what is the iterative procedure for integrating findings from statistical models with in-depth case studies? How do insights from each methodological phase inform and refine the other?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

## 3.5_validity_triangulation

In [ ]:
# Enter your query here
user_query = "In mixed-methods regional research, what qualitative techniques are used to probe and validate findings from quantitative spatial models? What specific methods help researchers understand mechanisms behind statistical patterns?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In case study research on regional economic transitions, how do researchers validate their theoretical explanations and findings? What role do local key informants play in ensuring the plausibility of research conclusions?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "In comparative qualitative research, what procedures ensure reliability when coding interview or archival data? How do multiple researchers check for agreement and resolve coding discrepancies?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

## Query visualisation

---



In [ ]:
# Enter your query here
user_query = "In studies of regional economic development paths, how are sequence index plots used to visualize how different regions evolve over time? What are the different ways to organize these plots (random order, sorted by initial state,or sorted by final state)?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "What visualization methods best support comparative analysis of multiple regional development trajectories? How do cluster visualizations help identify groups of regions with similar economic evolution patterns?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")

In [ ]:
# Enter your query here
user_query = "How should coordination effectiveness theoretically affect business formation?"  # ← Change this!

print(f"🔍 Query: {user_query}\n")

response = query_engine.query(user_query)

print("="*80)
print("📝 Response:")
print("="*80)
print(response.response)
print("="*80)

print("\n📚 Sources:")
for i, node in enumerate(response.source_nodes, 1):
    metadata = node.node.metadata
    score = node.score
    print(f"  {i}. {metadata.get('source', 'Unknown')} (Page {metadata.get('page', 'N/A')}) [Score: {score:.3f}]")